#Очистка данных

In [ ]:
import pandas as pd

In [ ]:
!pip install accelerate>=0.20.1

In [ ]:
!pip install transformers[torch]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 46.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 76.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 77.1 MB/s eta 0:00:00


In [ ]:
import accelerate
import transformers
from transformers import AutoTokenizer, AutoModelForTokenClassification, DataCollatorForTokenClassification, TrainingArguments, Trainer

In [ ]:
data = pd.read_excel("/content/Датасет.xlsx")

In [ ]:
data.isna().sum()

id                                             0
name(название)                                 0
specialization(специализация)                  0
responsibilities(Должностные обязанности)      0
requirements(Требования к соискателю)          0
terms(Условия)                                 0
skills(Ключевые навыки)                      996
salary_from                                    0
salary_to                                      0
object                                       992
city                                           0
address                                        2
type_of_employment_id(Тип занятости)           0
expirience(опыт работы)                       11
langs(знание языков)                           0
rights_id                                      0
notes(Примечания)                              0
employer_id                                    0
status_id                                      0
status_date                                    0
dadata_object       

In [ ]:
def contains(str1, str2):
  if str1.find(str2) != -1:
    return True
  else:
    return False
def clear_nan(string):
  if str(string) == "nan":
    return ""
  else:
    return str(string)

In [ ]:
data["requirements(Требования к соискателю)"] = data.apply(lambda x: clear_nan(x["requirements(Требования к соискателю)"]), axis=1)
data["terms(Условия)"] = data.apply(lambda x: clear_nan(x["terms(Условия)"]), axis=1)
data["notes(Примечания)"] = data.apply(lambda x: clear_nan(x["notes(Примечания)"]), axis=1)

In [ ]:
data["Содержит требования"] = data.apply(lambda x: contains(x["responsibilities(Должностные обязанности)"], x["requirements(Требования к соискателю)"]), axis=1)
data["Содержит условия"] = data.apply(lambda x: contains(x["responsibilities(Должностные обязанности)"], x["terms(Условия)"]) , axis=1)
data["Содержит примечания"] = data.apply(lambda x: contains(x["responsibilities(Должностные обязанности)"], x["notes(Примечания)"]) , axis=1)

In [ ]:
data_v1 = data[(data["Содержит примечания"]) & (data["Содержит условия"]) & (data["Содержит требования"])]

#Маркирование текста

In [ ]:
import re
regex = re.compile(r"[^ 0123456789а-яА-Я]")

In [ ]:
string = " .абв?10"
regex.sub("", string)

' абв10'

In [ ]:
data_v1["Clear responsibilities"] = data_v1.apply(lambda x: regex.sub("", x["responsibilities(Должностные обязанности)"]), axis=1)
data_v1["Clear requirements"] = data_v1.apply(lambda x: regex.sub("", x["requirements(Требования к соискателю)"]), axis=1)
data_v1["Clear terms"] = data_v1.apply(lambda x: regex.sub("", x["terms(Условия)"]), axis=1)
data_v1["Clear notes"] = data_v1.apply(lambda x: regex.sub("", x["notes(Примечания)"]), axis=1)

<ipython-input-12-607c396c4acb>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_v1["Clear responsibilities"] = data_v1.apply(lambda x: regex.sub("", x["responsibilities(Должностные обязанности)"]), axis=1)
<ipython-input-12-607c396c4acb>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_v1["Clear requirements"] = data_v1.apply(lambda x: regex.sub("", x["requirements(Требования к соискателю)"]), axis=1)
<ipython-input-12-607c396c4acb>:3: SettingWithCopyWarning: 
A value is trying to be set on a cop

In [ ]:
data_v1["Содержит требования"] = data_v1.apply(lambda x: contains(x["Clear responsibilities"], x["Clear requirements"]), axis=1)
data_v1["Содержит условия"] = data_v1.apply(lambda x: contains(x["Clear responsibilities"], x["Clear terms"]) , axis=1)
data_v1["Содержит примечания"] = data_v1.apply(lambda x: contains(x["Clear responsibilities"], x["Clear notes"]) , axis=1)

<ipython-input-13-53a5841fb500>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_v1["Содержит требования"] = data_v1.apply(lambda x: contains(x["Clear responsibilities"], x["Clear requirements"]), axis=1)
<ipython-input-13-53a5841fb500>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_v1["Содержит условия"] = data_v1.apply(lambda x: contains(x["Clear responsibilities"], x["Clear terms"]) , axis=1)
<ipython-input-13-53a5841fb500>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of 

In [ ]:
labels = ["requirements", "terms", "notes"]

In [ ]:
def get_tokens_with_entities(text, subtexts):
  spaces = [m.start() for m in re.finditer(' ', text)]
  starts = []
  strings = []
  for label in labels:
    string = subtexts[label]
    strings.append(string)
    starts.append(text.find(string))
  tags = "0"*len(text)
  for ind, i in enumerate(starts):
    tags = tags[:i] + str(ind+1)*len(strings[ind]) + tags[i + len(strings[ind]):]
  t_list = list(tags)
  for i in spaces:
    t_list[i] = " "
  tags = "".join(t_list)
  tokens_list = text.split(" ")
  tags_list = tags.split(" ")
  tokens = []
  tags = []
  for ind, i in enumerate(tokens_list):
    if i != "":
      tokens.append(i)
      tags.append(tags_list[ind])
  good_tags = []
  for tag in tags:
    good_tags.append(int(tag[0]))
  output = []
  for ind, i in enumerate(tokens):
    label_ind = good_tags[ind] - 1
    label = "O"
    if label_ind != -1:
      label = labels[label_ind]
    cort = ()
    if label == "O":
      cort = (i, label)
    else:
      if ind == 0:
        cort = (i, "B-" + label)
      else:
        if output[ind-1][1] != "O":
          if output[ind-1][1][2:] == label:
            cort = (i, "I-" + label)
          else:
            cort = (i, "B-" + label)
        else:
          cort = (i, "B-" + label)
    output.append(cort)
  return output

In [ ]:
subtexts = []
for ind in range(len(data_v1)):
  subtext = {}
  subtext["requirements"] = data_v1.iloc[ind]["Clear requirements"]
  subtext["terms"] = data_v1.iloc[ind]["Clear terms"]
  subtext["notes"] = data_v1.iloc[ind]["Clear notes"]
  subtexts.append(subtext)

In [ ]:
subtexts[0]

{'requirements': '',
 'terms': '',
 'notes': 'Работа на строительных площадках Очистных сооружениях'}

In [ ]:
marked = []
for ind in range(len(data_v1)):
  vacancy = data_v1.iloc[ind]
  entities = get_tokens_with_entities(vacancy["Clear responsibilities"], subtexts[ind])
  marked.append(entities)

In [ ]:
len(subtexts)

959

In [ ]:
marked[0]

[('Работа', 'B-notes'),
 ('на', 'I-notes'),
 ('строительных', 'I-notes'),
 ('площадках', 'I-notes'),
 ('Очистных', 'I-notes'),
 ('сооружениях', 'I-notes')]

#Model

In [ ]:
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.2/486.2 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 22.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 39.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 31.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 19.9 MB/s eta 0:00:00


In [ ]:
class NERDataMaker:
    def __init__(self, texts, entities):
        self.unique_entities = []
        self.processed_texts = []

        temp_processed_texts = []
        for ind, text in enumerate(texts):
            tokens_with_entities = get_tokens_with_entities(text, entities[ind])
            for _, ent in tokens_with_entities:
                if ent not in self.unique_entities:
                    self.unique_entities.append(ent)
            temp_processed_texts.append(tokens_with_entities)

        self.unique_entities.sort(key=lambda ent: ent if ent != "O" else "")

        for tokens_with_entities in temp_processed_texts:
            self.processed_texts.append([(t, self.unique_entities.index(ent)) for t, ent in tokens_with_entities])

    @property
    def id2label(self):
        return dict(enumerate(self.unique_entities))

    @property
    def label2id(self):
        return {v:k for k, v in self.id2label.items()}

    def __len__(self):
        return len(self.processed_texts)

    def __getitem__(self, idx):
        def _process_tokens_for_one_text(id, tokens_with_encoded_entities):
            ner_tags = []
            tokens = []
            for t, ent in tokens_with_encoded_entities:
                ner_tags.append(ent)
                tokens.append(t)

            return {
                "id": id,
                "ner_tags": ner_tags,
                "tokens": tokens
            }

        tokens_with_encoded_entities = self.processed_texts[idx]
        if isinstance(idx, int):
            return _process_tokens_for_one_text(idx, tokens_with_encoded_entities)
        else:
            return [_process_tokens_for_one_text(i+idx.start, tee) for i, tee in enumerate(tokens_with_encoded_entities)]

    def as_hf_dataset(self, tokenizer):
        from datasets import Dataset, Features, Value, ClassLabel, Sequence
        def tokenize_and_align_labels(examples):
            tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)

            labels = []
            for i, label in enumerate(examples[f"ner_tags"]):
                word_ids = tokenized_inputs.word_ids(batch_index=i)  # Map tokens to their respective word.
                previous_word_idx = None
                label_ids = []
                for word_idx in word_ids:  # Set the special tokens to -100.
                    if word_idx is None:
                        label_ids.append(-100)
                    elif word_idx != previous_word_idx:  # Only label the first token of a given word.
                        label_ids.append(label[word_idx])
                    else:
                        label_ids.append(-100)
                    previous_word_idx = word_idx
                labels.append(label_ids)

            tokenized_inputs["labels"] = labels
            return tokenized_inputs

        ids, ner_tags, tokens = [], [], []
        for i, pt in enumerate(self.processed_texts):
            ids.append(i)
            pt_tokens,pt_tags = list(zip(*pt))
            ner_tags.append(pt_tags)
            tokens.append(pt_tokens)
        data = {
            "id": ids,
            "ner_tags": ner_tags,
            "tokens": tokens
        }
        features = Features({
            "tokens": Sequence(Value("string")),
            "ner_tags": Sequence(ClassLabel(names=dm.unique_entities)),
            "id": Value("int32")
        })
        ds = Dataset.from_dict(data, features)
        tokenized_ds = ds.map(tokenize_and_align_labels, batched=True)
        return tokenized_ds

In [ ]:
dm = NERDataMaker(list(data_v1["Clear responsibilities"]), subtexts)

In [ ]:
print(f"total examples = {len(dm)}")
print(dm[0:3])

total examples = 959
[{'id': 0, 'ner_tags': [1, 4, 4, 4, 4, 4], 'tokens': ['Работа', 'на', 'строительных', 'площадках', 'Очистных', 'сооружениях']}, {'id': 1, 'ner_tags': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'tokens': ['Гнутье', 'арматурной', 'стали', 'на', 'механических', 'станках', 'при', 'количестве', 'отгибов', 'на', 'одном', 'стержне', 'более', 'четырех', 'Сборка', 'и', 'установка', 'сеток', 'и', 'плоских']}, {'id': 2, 'ner_tags': [0, 0], 'tokens': ['Вязка', 'арматуры']}]


In [ ]:
"""tokenizer = AutoTokenizer.from_pretrained("yqelz/xml-roberta-large-ner-russian")
data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)
model = AutoModelForTokenClassification.from_pretrained("yqelz/xml-roberta-large-ner-russian", num_labels=len(dm.unique_entities), id2label=dm.id2label, label2id=dm.label2id)"""
tokenizer = AutoTokenizer.from_pretrained("Evolett/rubert-tiny2-finetuned-ner")
data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)
model = AutoModelForTokenClassification.from_pretrained("Evolett/rubert-tiny2-finetuned-ner", num_labels=len(dm.unique_entities), id2label=dm.id2label, label2id=dm.label2id, ignore_mismatched_sizes=True)

Some weights of BertForTokenClassification were not initialized from the model checkpoint at Evolett/rubert-tiny2-finetuned-ner and are newly initialized because the shapes did not match:
- classifier.weight: found shape torch.Size([9, 312]) in the checkpoint and torch.Size([7, 312]) in the model instantiated
- classifier.bias: found shape torch.Size([9]) in the checkpoint and torch.Size([7]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
training_args = TrainingArguments(
    output_dir="/content/results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=20,
    weight_decay=0.01,
)

train_ds = dm.as_hf_dataset(tokenizer=tokenizer)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=train_ds, # eval on training set! ONLY for DEMO!!
    tokenizer=tokenizer,
    data_collator=data_collator,
)

Map:   0%|          | 0/959 [00:00<?, ? examples/s]

In [ ]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,No log,0.629804
2,No log,0.591859
3,No log,0.558687
4,No log,0.537340
5,No log,0.526513
6,No log,0.500053
7,No log,0.484845
8,No log,0.467233
9,0.581300,0.452108
10,0.581300,0.438991


TrainOutput(global_step=1200, training_loss=0.5120942179361979, metrics={'train_runtime': 135.4423, 'train_samples_per_second': 141.61, 'train_steps_per_second': 8.86, 'total_flos': 88404320801940.0, 'train_loss': 0.5120942179361979, 'epoch': 20.0})

In [ ]:
from transformers import pipeline
pipe = pipeline("ner", model=model, tokenizer=tokenizer, aggregation_strategy="simple", device=0)

In [ ]:
def convert_result(result):
  requirements = ""
  terms = ""
  notes = ""
  for span in result:
    label = span["entity_group"]
    word = span["word"]
    if label == "requirements":
      requirements += word + " "
    elif label == "terms":
      terms += word + " "
    elif label == "notes":
      notes += word + " "
  return requirements, terms, notes

In [ ]:
input_index = 67
input = data_v1.iloc[input_index]["Clear responsibilities"]
input

'В строительную компанию требуется штукатурымаляры возможно готовые бригады  Требования   Опыт работы от 1 года   Дисциплина   Ответственность  Обязанности   Внутренние отделочные работы ТРЦ  Условия   Еженедельные выплаты   Без задержек ЗП   Адекватное руководство   Инструмент наш с вас только желание работать и зарабатывать  Если у вас есть опыт и вы ищете стабильность и постоянство  звоните или откликайтесь на вакансию'

In [ ]:
result = pipe(input)
convert_result(result)

('Опыт работы от 1 года Дисциплина Ответственность Внутрен опыт ',
 'Еже ##недельные выплаты Без задержек ЗП Адекватное руководство Инструмент наш с вас только желание работать и зарабатывать Если у вас есть и вы ищете стабильность и постоянство звоните или откликайтесь на вакансию ',
 '')

In [ ]:
pd.DataFrame(data_v1.iloc[input_index][["Clear responsibilities", "Clear requirements", "Clear terms", "Clear notes"]])

,94
Clear responsibilities,В строительную компанию требуется штукатурымал...
Clear requirements,Опыт работы от 1 года Дисциплина Ответстве...
Clear terms,Еженедельные выплаты Без задержек ЗП Адекв...
Clear notes,


In [ ]:
test = pipe("На стройку требуются МОНТАЖНИКИ, РАЗНОРАБОЧИЕ  Срочно набираем сотрудников в города на выбор: Керчь, Тольятти, Казань, Ростов-на-Дону, Нижнекамск, Сызрань, Москва, Курган!!!  Условия:  - БЕСПЛАТНЫЙ комплексный горячий обед.  - БЕСПЛАТНОЕ проживание в гостинице.  - БЕСПЛАТНЫЙ корпоративный транспорт.  - БЕСПЛАТНАЯ спецодежда.  - Есть дневные и ночные.  - 55 000 рублей в месяц (!!чистыми!!)  - Вахта ОТ 30 ДНЕЙ и выше.  - Официальное трудоустройство по ТК РФ (с занесением стажа в трудовую книжку).  - Компенсация проезда для иногородних.  Требования:  - Желателен опыт работы с болгаркой  Обязанности:  - Работа в паре со сварщиком, резка, вязка арматуры, зачистка сварочных швов болгаркой.  - Работы с бетоном и другие подсобные работы.  В нашей компании действует акция &quot;ПРИВЕДИ ДРУГА&quot;, премия 4000 рублей (если вы приезжаете с другом на работу и он отрабатывает не менее 10 смен). Можете сообщить знакомым и друзьям, о наборе специалистов в нашу компанию!  Ждем Вас!")
convert_result(test)

(': - Желателен опыт работы с болгаркой Обязанности : - Работа в паре со сварщиком ##ка ##ка арматуры ##чных болгаркой бетоном не ',
 ': - БЕСПЛАТНЫЙ комплексный горячий обед. - БЕСПЛАТНОЕ проживание в гостинице. - БЕСПЛАТНЫЙ корпоративный транспорт. - БЕСПЛАТНАЯ спецодежда. - Есть дневные и ночные. - 55 000 рублей в месяц (!! чистыми!! ) Вахта 30 ДНЕЙ и выше. - Официальное трудоустройство по ТК РФ ( с занесением стажа в трудовую книжку ). - Компенсация проезда для иногородних. ',
 'компании действует акция ПР ##ВЕДИ ##РУ ##t ;, премия 4000 рублей ( если вы приезжаете с другом на работу и отрабатывает менее 10 смен , о наборе специалистов компанию ')